In [ ]:
import ipywidgets
from ipywidgets import interact

In [ ]:
def f(x):
    return x

In [ ]:
interact(f,x=10)

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import numpy as np
import pandas as pd
    
from pathlib import Path #we'll import Path object with `.ls` method added later
from pprint import pprint as pp

import pdb

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr

from holoviews import opts
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import Stream, param
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts


# import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf

hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'

# Dashboards
import param as pm, panel as pn
pn.extension()

In [ ]:
# Geoviews visualization default options
H,W, = 250,250
opts.defaults(
    opts.RGB(height=H, width=W, tools=['hover'], active_tools=['wheel_zoom']),
    opts.Image(height=H, width=W, tools=['hover'], active_tools=['wheel_zoom'], framewise=True),#axiswise=True ),
    opts.Points( tools=['hover'], active_tools=['wheel_zoom']),
)

## Set up additional library path

In [ ]:
# Add the utils directory to the search path
SP_ROOT = Path.home()/'Playground/ContextNet'
SP_LIBS = SP_ROOT/'scripts' # to be changed to 'src'
# LIBS_DIR = Path('../src').absolute()
DIRS_TO_ADD = [SP_LIBS]#, LIBS_DIR]
for p in DIRS_TO_ADD:
    assert p.exists()
    
    if str(p) not in sys.path:
        sys.path.insert(0, str(p))
        print(f"Added to sys.path: {p}")

# pp(sys.path)
    

In [ ]:
from output_helpers import print_mro as mro, nprint, Path
import SpacenetPath as spp
import spacenet_globals as spg
# from output_helpers import Path #.ls method is added to Path class

## Step 1: Explore your dataset
    

In [ ]:
city = 'vegas'
rgb8_dir = spp.sample_rgb8_dirs[city]
mask_dir = spp.sample_mask_dirs[city]
sp_vec_dir = spp.sample_road_vec_dirs[city]
osm_mask_dir = spp.sample_mask_dirs[city]

In [ ]:
rgb_fns = sorted([rgb8_dir/fn for fn in rgb8_dir.ls() if Path(fn).suffix in ['.tif', '.tiff']])
mask_fns = sorted([mask_dir/fn for fn in mask_dir.ls() if Path(fn).suffix in ['.tif', '.tiff']])

In [ ]:
for rgb_fn, mask_fn in zip(rgb_fns, mask_fns):
    assert rgb_fn.exists() and mask_fn.exists()
    



## Playground with `param` library

In [ ]:
# class Explorer(pm.Parameterized):

In [ ]:
# pn.panel(rgb_fns[0])

In [ ]:
import xarray as xr
from rasterio.plot import reshape_as_image

In [ ]:
rgb = xr.open_rasterio(rgb_fns[0])

In [ ]:
rgb_arr = reshape_as_image(rgb.data)
# plt.imshow(rgb_arr)

In [ ]:
import osmnx as ox
import rasterio as rio

class A(pm.Parameterized):
    fn = pm.FileSelector(path=str(rgb8_dir/"*.tif"))
   
    osm_log = pm.String(default="", 
                        label="OSM log",
                        doc="simple log string about OSM download status")
    osm_count = pm.Number(0)#, precedence=0) #inivisible widget
    action = pm.Action(lambda x: x.param.trigger('action'), label='click to download osm')

    
    @param.depends('action')
    def update_osm(self):
        print('Started downloading osm data')
        print("OSM data downloaded")
        self.osm_count += 1
#         self.osm = ...

    
# import osmnx as ox
# import rasterio as rio

# class A(pm.Parameterized):
#     fn = pm.FileSelector(path=str(rgb8_dir/"*.tif"))
#     action = pm.Action(lambda x: x.param.trigger('action'),
#                           doc="Download OSM road network data in the current RGB file's bounds")
    
#     # todo: this is not really a parameter. I should have a dynamicmap element that displays logs
#     # in repsonse to the download_osm method's success code
#     osm_log = pm.String(default="", 
#                         label="OSM log",
#                         doc="simple log string about OSM download status")
#     osm_dl_count = pm.Integer(0)#, precedence=0) #inivisible widget
    
#     def __init__(self):
#         self.gv_osm = hv.Div("")
    
#     @param.depends('action')
#     def dl_and_update_osm():
#         print('started to dl osm data')
#         with rio.open(self.fn) as ds:
#             bounds = ds.bounds
#         north, south, east, west = bounds.top, bounds.bottom, bounds.right, bounds.left
#         G = ox.graph_from_bbox(north, south, east, west)
#         self.osm_log = f'OSM data downloaded: {bounds}'
#         self.osm_dl_count += 1

#         print("OSM data downloaded")
        
#         self.gv_osm = gv.Path(ox.graph_to_gdfs(G, edges=True, nodes=False))
#         return
    

                           
                           
                           
                           
     

In [ ]:
a1 = A()
pn.Column(a1.param).servable()


In [ ]:
pn.panel(a1.param.osm_count)
a1.osm_count

In [ ]:
class ActionExample(param.Parameterized):
    """
    Demonstrates how to use param.Action to trigger an update.
    """

    number = param.Integer(default=0)
    
    action = param.Action(lambda x: x.param.trigger('action'), label='Download OSM!')
        
    @param.depends('action')
    def get_number(self):
        print('action triggered')
        self.number += 1
#         return self.number
# todo add init function and see if it resolves this issue
# Then, ask questions about when the parameters stay as class variables vs. instance variable
# - link to the Topica manual

In [ ]:
action_example = ActionExample()
# pn.Column(
#     '# param.Action Example',
#     pn.Row(
#         pn.Column(pn.panel(action_example.param, show_labels=False, show_name=False, margin=0),
#             'Click the button to trigger an update in the output.'),
#         pn.WidgetBox(action_example.get_number, width=300))
# ).servable()
                                
pn.Column(pn.panel(action_example.param, show_labels=False, show_name=False, margin=0),
            'Click the button to trigger an update in the output.',
                 pn.WidgetBox(action_example.get_number, width=300)
).servable()

In [ ]:
pn.panel(action_example.param.number)        

In [ ]:
class Example(param.Parameterized):
    fn = param.FileSelector(path=str(rgb8_dir/"*.tif"))
    action = param.Action(lambda x: x.param.trigger('action'),
                       label='Download OSM')
    
    osm_count = param.Number(default=0.)#, precedence=0) #inivisible widget
    
    def __init__(self,**kwargs):
#         pdb.set_trace()
        super().__init__(**kwargs)
        print('created object')
    
    @param.depends('action', watch=True)
    def update_osm(self):
        print('Started downloading osm data')
        print("OSM data downloaded")
        self.osm_count += 1
#         self.osm = ...


In [ ]:
ex = Example()

In [ ]:
ex.update_osm()
print(ex.osm_count)

In [ ]:
pn.Column(ex.param)

In [ ]:
pn.panel(ex.param.osm_count)

In [ ]:
ex.osm_count

In [ ]:
ex.param['action']

## Template 

In [ ]:
CITIES = ['LA', 'BOSTON', 'PARIS', 'JEONJU', 'DC', 'SF']
class MyParamedClass(pm.Parameterized):
    
    # Declare parameters (which will turned into instance attributes via. pm.Parametrized.__init__ method)
    age = pm.Integer(10, bounds=(1,100))
    city = pm.Selector(objects=CITIES, default='LA')
    logbox = pm.String('LOGBOX')
    
    # Explicitly define initialization method
    ## This will be called to instantiate new instances for this class
    def __init__(self, **params):
        """
        The signature of this init method matched pm.Parametrized class's init method:
            pm.Parametrized.__init__(self, **params)
        This allows to specify parameter values at instaniation using 'key'=value: 
            eg: me = MyParamedClass(age=11, city='JEONJU')
            
        See pm.Parametrized?? for details
        """
        super().__init__(**params)
        self.country = 'USA'
        
    # Annotate a method with its dependencies that does not need automatic update when the dependents' values change
    @param.depends('age', watch=False) # watch=False is default
    def get_stage(self):
        stage = 'old' if self.age > 50 else 'young'
        return stage
    
    # Annotate method that needs be updated automatically as its dependents' parameter values change
    @param.depends('city', watch=True)
    def update_country(self):
        if self.city in ['LA', 'BOSTON', 'DC', 'SF']:
            self.country = 'USA'
        elif self.city == 'PARIS':
            self.country = 'FRANCE'
        else:
            self.country = 'KOREA'
        print(f"Country is updated: {self.country}")
        
        # Just for demonstrating that this method is actually called when `city` parameter changes
        self.logbox = self.country
        print('Logbox: ', self.logbox)
        
                
    
    
        
    

In [ ]:
myex = MyParamedClass()

In [ ]:
pn.panel(myex.param)#.servable()

In [ ]:
pn.panel(myex.param['logbox'])